# 3. 基本库的使用

## 3.1 urllib

urllib 是 Python 内置的 HTTP 请求库，包含四个模块：
* request: 它是最基本的 HTTP 请求模块，可以用来模拟发送请求。
* error: 异常处理模块，如果出现请求错误，我们可以捕获这些异常。
* parse: 工具模块，提供许多 URL 处理方法，比如拆分、解析、合并等。
* robotparser: 主要是用来识别网站的 robots.txt 文件，判断哪些网页可以爬，哪些不可以爬。

### 3.1.1 发送请求

使用 urllib 的 request 模块，可以方便地实现请求的发送并得到响应。

1. urlopen()

In [ ]:
import urllib.request

response = urllib.request.urlopen('https://www.python.org')
print(response.read().decode('utf-8'))  # read() 可以得到返回的网页内容
print(type(response))  # <class 'http.client.HTTPResponse'>
print(response.status)  # 200
print(response.getheaders())

也可以给链接传递一些参数，urllib()函数的 API:
```
urllib.request.urlopen(url, data=None, [timeout,]*, cafile=None, capath=None, cadefault=False, context=None)
```
* data 参数：可选，需要使用 bytes() 将参数转化为 bytes 类型。另外，需使用 POST 请求方式。

In [5]:
import urllib.parse
import urllib.request

data = bytes(urllib.parse.urlencode({'word': 'hello'}), encoding='utf-8')
response = urllib.request.urlopen('http://httpbin.org/post', data=data)
print(response.read().decode('utf-8'))

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "word": "hello"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Content-Length": "10", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "Python-urllib/3.7"
  }, 
  "json": null, 
  "origin": "114.240.125.16, 114.240.125.16", 
  "url": "https://httpbin.org/post"
}



可见，传递的参数出现在 form 字段中，这表明是模拟了表单提交的方式，以 POST 方式传输数据。
* timeout 参数：设置超时时间，单位为秒。如果超时无响应，会抛异常。默认使用全局默认时间。

In [ ]:
import urllib.request

response = urllib.request.urlopen('http://httpbin.org/get', timeout=1)

* 其它参数: context 参数，必须是 ssl.SSLContext 类型，用来指定 SSL 设置，cafile, capath 分别指定 CA 证书和它的路径，在请求 HTTPS 链接时有用，cadefault 参数已弃用。

2. Request

如果请求中需要加入 Headers 等信息，可以利用更强大的 Request 类来构建。Request 的构造方法如下：
```
class urllib.request.Request(url, data=None, headers={}, origin_req_host=None, unverifiable=False, method=None)
```
* url：请求 url，必传参数，其它都是可选参数；
* data：如果要传，必须是 bytes 类型，可用 urllib.parse 模块里的 urlencode() 进行编码；
* headers：请求头，可以在构造请求时通过 headers 参数直接构造，也可以通过调用请求实例的 add_header() 方法构造；添加请求头最常用的用法就是通过修改 User-Agent 来伪装浏览器；
* origin_req_host：请求方的 host 名称或 IP 地址；
* unverifiable：表示这个请求是否是无法通过验证的，默认是 False；
* method：请求方法，比如 GET、POST、和 PUT 等。

In [4]:
from urllib import request, parse

url = 'http://httpbin.org/post'
headers = {
    'User-Agent': 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)',
    'Host': 'httpbin.org'
}
dict = {
    'name': 'Germey'
}
data = bytes(parse.urlencode(dict), encoding='utf-8')
req = request.Request(url=url, data=data, headers=headers, method='POST')
response = request.urlopen(req)
print(response.read().decode('utf-8'))

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "name": "Germey"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Content-Length": "11", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)"
  }, 
  "json": null, 
  "origin": "114.240.125.16, 114.240.125.16", 
  "url": "https://httpbin.org/post"
}



3. 高级用法 

介绍两种高级类，一是 Handler，可以理解为各种处理器，有处理登陆验证的，有处理 Cookies 的，有处理代理设置的。urllib.request 模块里的 BaseHandler 类是其它 Handler 的父类，它提供了最基本的用法，例如 default_open()、protocol_request() 等。一些子类如下：
* 

### 3.1.2 处理异常